## 1 Ano de dados

In [1]:
url = "http://vps-40d69db1.vps.ovh.ca:23002/api/CotacaoIntradayAcaoResultadoValors/obtemDatasPorDataGrupoLimite?diaInicioNum=20210301&diaFinalNum=20220228&grupoAcaoId=12&limiteDiaRegra=260"

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [3]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [4]:
df.head()

,anoMesNum,ticker,codigoRegraProjecao,target,stop,totalLucro,totalPrejuizo,mediaDias
0,202103,ABEV3,t09s15vy,0.09,0.15,0,345,41.9449
1,202103,ABEV3,t10s15vy,0.10,0.15,0,345,41.9449
2,202103,ABEV3,t10s17vy,0.10,0.17,0,345,47.0261
3,202103,ABEV3,t12s15vy,0.12,0.15,0,345,41.9449
4,202103,ABEV3,t12s17vy,0.12,0.17,0,345,47.0261


## Cálculo de Saldo

In [5]:
df['saldo'] =(df['target'] * df['totalLucro']) - (df['stop'] * df['totalPrejuizo'])

In [6]:
df.head()

,anoMesNum,ticker,codigoRegraProjecao,target,stop,totalLucro,totalPrejuizo,mediaDias,saldo
0,202103,ABEV3,t09s15vy,0.09,0.15,0,345,41.9449,-51.75
1,202103,ABEV3,t10s15vy,0.10,0.15,0,345,41.9449,-51.75
2,202103,ABEV3,t10s17vy,0.10,0.17,0,345,47.0261,-58.65
3,202103,ABEV3,t12s15vy,0.12,0.15,0,345,41.9449,-51.75
4,202103,ABEV3,t12s17vy,0.12,0.17,0,345,47.0261,-58.65


In [8]:
dfABEV = df[df['ticker'] == 'ABEV3']

In [9]:
dfABEV

,anoMesNum,ticker,codigoRegraProjecao,target,stop,totalLucro,totalPrejuizo,mediaDias,saldo
0,202103,ABEV3,t09s15vy,0.09,0.15,0,345,41.9449,-51.75
1,202103,ABEV3,t10s15vy,0.10,0.15,0,345,41.9449,-51.75
2,202103,ABEV3,t10s17vy,0.10,0.17,0,345,47.0261,-58.65
3,202103,ABEV3,t12s15vy,0.12,0.15,0,345,41.9449,-51.75
4,202103,ABEV3,t12s17vy,0.12,0.17,0,345,47.0261,-58.65
...,...,...,...,...,...,...,...,...,...
8582,202202,ABEV3,t12s17vy,0.12,0.17,49,0,10.9388,5.88
8583,202202,ABEV3,t13s17vy,0.13,0.17,53,0,14.1321,6.89
8584,202202,ABEV3,t14s17vy,0.14,0.17,19,0,13.5263,2.66
8585,202202,ABEV3,t15s12vy,0.15,0.12,8,0,13.0000,1.20


In [16]:
dfAgrupado = df\
        .groupby(['anoMesNum','codigoRegraProjecao','target','stop']).sum(['totalLucro','totalPrejuizo'])\
        .filter(items=['totalLucro','totalPrejuizo'])

In [22]:
dfAgrupado = dfAgrupado.reset_index()

In [23]:
dfAgrupado['saldo'] =(dfAgrupado['target'] * dfAgrupado['totalLucro']) - (dfAgrupado['stop'] * dfAgrupado['totalPrejuizo'])

In [26]:
dfAgrupado['total'] = dfAgrupado['totalLucro'] + dfAgrupado['totalPrejuizo']

In [28]:
dfAgrupado.head()

,anoMesNum,codigoRegraProjecao,target,stop,totalLucro,totalPrejuizo,saldo,total
0,202103,t09s15vy,0.09,0.15,8944,13367,-1200.09,22311
1,202103,t10s15vy,0.10,0.15,11571,19637,-1788.45,31208
2,202103,t10s17vy,0.10,0.17,12564,18625,-1909.85,31189
3,202103,t12s15vy,0.12,0.15,10308,20700,-1868.04,31008
4,202103,t12s17vy,0.12,0.17,11295,19571,-1971.67,30866


In [32]:
dfMelhores = df\
            .filter(items=['codigoRegraProjecao','saldo'])\
            .groupby('codigoRegraProjecao').agg({'saldo' : np.sum })

In [34]:
dfMelhores.sort_values('saldo')

,saldo
codigoRegraProjecao,
t09s15vy,4356.39
t10s15vy,5733.95
t12s15vy,6357.69
t15s12vy,6406.95
t10s17vy,6637.61
t12s17vy,7895.28
t15s15vy,8006.85
t13s17vy,8584.74
t14s17vy,9082.72
